In [1]:
from tensorflow import keras
from numpy import load
import json
from pandas import read_csv,DataFrame,merge
from ast import literal_eval
from os import path

### STORE
o_player_sequences, _ = load(f'../Notebooks/trajectories.npy',allow_pickle=True) # All of the Offensive Player Sequences in a Numpy Array

### STORE
sequences_ordered = load(f'../Notebooks/sequences_ordered_full.npy',allow_pickle=True) # All of the ordered cluster sequences 

# The game play o_player d_player id's corresponding to each sequence in o_player_sequences
### STORE
o_player_sequences_id = load(f'../Notebooks/o_player_sequences_id.npy',allow_pickle=True) 

# NOT USED!! 
# with open(f'../Notebooks/player_pairs.json', 'r') as f: # The pairs of players grouped together for a given Game,Play
#    player_pairs_str = json.load(f)
#    player_pairs = {literal_eval(key_str): value for key_str, value in player_pairs_str.items()}


# Dictionary of all o_player_sequence indexes that correspond to a Game Play key
with open(f'../Notebooks/player_pair_ids.json', 'r') as f: 
    player_pair_ids_str = json.load(f)
    player_pair_ids = {literal_eval(key_str): value for key_str, value in player_pair_ids_str.items()}

with open('../Notebooks/all_trajectory_dict.json', 'r') as f:
    all_trajectory_dict_1_json = json.load(f)
    all_trajectory_dict = {literal_eval(key_str): value for key_str, value in all_trajectory_dict_1_json.items()}
player_pair_ids = {k:v for k,v in player_pair_ids.items() if v}

### STORE
player_pair_ids = {k:v for k,v in player_pair_ids.items() if v} 

trajectory_dict_keys = DataFrame(list(player_pair_ids.keys()),columns=["gameId","playId"])
bdb_games = read_csv(f'../NFLData/all_games.csv')
bdb_plays = read_csv(f'../NFLData/all_plays.csv')
bdb_players = read_csv(f'../NFLData/all_players.csv')

valid_games = trajectory_dict_keys[["gameId"]].drop_duplicates() 
valid_plays = trajectory_dict_keys[["gameId","playId"]].drop_duplicates()
valid_games = merge(bdb_games,valid_games,on="gameId",how="inner")

###STORE
valid_plays = merge(bdb_plays,valid_plays,on=["gameId","playId"],how="inner")


cluster_options = [{'label': f'Cluster Center {i+1}', 'value': i} for i in range(8)]
cluster_center_options = [{'label': f'Sequence {i+1}', 'value': i} for i in range(8)]


### Store Game Options


In [2]:
valid_games.to_csv("all_valid_games.csv")
valid_plays.to_csv("all_valid_plays.csv")

In [7]:
print(player_pair_ids.keys())

dict_keys([(2021090900, 97), (2021090900, 410), (2021090900, 434), (2021090900, 456), (2021090900, 480), (2021090900, 509), (2021090900, 559), (2021090900, 583), (2021090900, 869), (2021090900, 925), (2021090900, 947), (2021090900, 1203), (2021090900, 1227), (2021090900, 1267), (2021090900, 1312), (2021090900, 2008), (2021090900, 2032), (2021090900, 2441), (2021090900, 2871), (2021090900, 2893), (2021090900, 2915), (2021090900, 3062), (2021090900, 3110), (2021090900, 3134), (2021090900, 3462), (2021090900, 3633), (2021090900, 3793), (2021090900, 3862), (2021090900, 3933), (2021090900, 3962), (2021090900, 4538), (2021090900, 4560), (2021090900, 4624), (2021090900, 4653), (2021090900, 4677), (2021090900, 4750), (2021090900, 4772), (2021091910, 54), (2021091910, 101), (2021091910, 125), (2021091910, 168), (2021091910, 325), (2021091910, 404), (2021091910, 451), (2021091910, 473), (2021091910, 734), (2021091910, 756), (2021091910, 976), (2021091910, 1025), (2021091910, 1290), (2021091910, 

### PREPROCESS DATA BEFORE LOADING INTO DASH APP!!!

### Sequence Document Structure:

- id
- seqIndex: int (index in o_player_sequences)
- seqGameID: Game ID of corresponding game
- seqPlayID: Play ID of corresponding play
- seqOPlayerID: Offensive player ID on the play
- seqDPlayerID: Defensive player ID on the play
- seqOArrX: X Array of the Offensive player 
- seqOArrY: Y Array of the Offensive player 
- seqDArrX: X Array of the actual Offensive player
- seqDArrY: Y Array of the actual Offensive player
- seqOTeamX: X Trajectories of all other Offensive players 
- seqOTeamY: Y Trajectories of all other players 
- seqDTeamX: Y Trajectories of all other players 
- seqDTeamY: Y Trajectories of all other players 

In [4]:
seq_doc_to_upload = []

for i in range(len(o_player_sequences)):
    play_key = o_player_sequences_id[i] # game play o_player d_player
    o_player_id = int(play_key[2])
    d_player_id = int(play_key[3])
    play = valid_plays[(valid_plays["gameId"] == play_key[0])&(valid_plays["playId"] == play_key[1])]
    game = valid_games[(valid_games["gameId"] == play_key[0])]
    h_team = game.iloc[0]['homeTeamAbbr']
    a_team = game.iloc[0]['visitorTeamAbbr']
    off_team = play.iloc[0]['possessionTeam']
    #def_team = game.iloc[0]['defensiveTeam']
    if h_team == off_team:
        def_team = a_team
    else:
        def_team = h_team
    actual_los = play.iloc[0]['absoluteYardlineNumber']
    off_play = all_trajectory_dict[(play_key[0],play_key[1],off_team)]
    def_play = all_trajectory_dict[(play_key[0],play_key[1],def_team)]
    off_play = {int(float(k)):v for k,v in off_play.items()}
    def_play = {int(float(k)):v for k,v in def_play.items()}
    o_player = off_play.pop(o_player_id,[])
    d_player = def_play.pop(d_player_id,[])
    
    if len(d_player) == 0 or len(o_player) == 0:
        continue
    off_seqs = list(off_play.values())
    def_seqs = list(def_play.values())
    off_seq_x = [x[0] for i in range(len(off_seqs)) for x in off_seqs[i]]
    off_seq_y = [x[1] for i in range(len(off_seqs)) for x in off_seqs[i]]
    def_seq_x = [x[0] for i in range(len(def_seqs)) for x in def_seqs[i]]
    def_seq_y = [x[1] for i in range(len(def_seqs)) for x in def_seqs[i]]
    
    original_o_player_x = [x[0] for x in o_player]
    original_o_player_y = [x[1] for x in o_player]

    original_d_player_x = [x[0] for x in d_player]
    original_d_player_y = [x[1] for x in d_player]
    seq_doc_to_upload.append({'seqIndex':i,'seqGameID':play_key[0],'seqPlayID':play_key[1],'seqOPlayerID':o_player_id,'seqDPlayerID':d_player_id,'seqOArrX':original_o_player_x,
                              'seqOArrY':original_o_player_y,'seqDArrX':original_d_player_x,'seqDArrY':original_d_player_y,'seqOTeamX':off_seq_x,'seqOTeamY':off_seq_y,
                              'seqDTeamX':def_seq_x,'seqDTeamY':def_seq_y,'seqOSeqFull':o_player_sequences[i]})
    

KeyError: (2018090600.0, 75.0, 'ATL')

In [7]:
import pymongo
from pymongo import MongoClient
client = MongoClient()



In [8]:
db = client.CPSC502
seq_collection = db.Sequence

In [5]:
seq_collection.insert_many(seq_doc_to_upload)

### Cluster Sequence Document Structure:

- id
- clusterID: The Center it belongs to (-1 for default showing)
- clusterSeqX: X array of the cluster 
- clusterSeqY: Y array of the cluster

In [3]:
cluster_doc_to_upload = []

for i in range(len(sequences_ordered)):
    current_c_index = sequences_ordered[i][0][1]
    seq = o_player_sequences[current_c_index]
    c_x = [x[0] for x in seq]
    c_y = [x[1] for x in seq]   
    cluster_doc_to_upload.append({'clusterID':-1,'clusterSeqX':c_x,'clusterSeqY':c_y,'clusterOriginalID':current_c_index,'clusterOrder':i})
    for j in range(8):
        current_c_index = sequences_ordered[i][j][1]
        seq = o_player_sequences[current_c_index]
        c_x = [x[0] for x in seq]
        c_y = [x[1] for x in seq]   
        cluster_doc_to_upload.append({'clusterID':i,'clusterSeqX':c_x,'clusterSeqY':c_y,'clusterOriginalID':current_c_index,'clusterOrder':j})


In [9]:
cluster_collection = db.Cluster
cluster_collection.insert_many(cluster_doc_to_upload)

### Game Options Document Structure:

- id
- goLabel: The label of the game option
- goValue: The value of the game option

In [9]:
game_options = []
for row in valid_games.itertuples():
    game_options.append({'label':f'{row.homeTeamAbbr} vs. {row.visitorTeamAbbr} playing home to {row.homeTeamAbbr} on {row.gameDate}','value':row.gameId})

In [10]:
game_option_collection = db.GameOption
game_option_collection.insert_many(game_options)